In [2]:
import torch

In [2]:
!git clone https://github.com/WongKinYiu/yolov7.git

fatal: destination path 'yolov7' already exists and is not an empty directory.


In [7]:
import sys
sys.path.append('./yolov7')

In [4]:
!pip install opencv-python

In [8]:
import torch
from models.yolo import Model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
#downloded the YOLOv7.pt from the official githup repo: use: wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
#https://github.com/WongKinYiu

In [9]:
# Model: YOLOv7 architechure 
model_arch = torch.load('yolov7.pt', map_location=device, weights_only=False)

In [8]:
print(model_arch)

{'model': Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_siz

In [9]:
# If it's a checkpoint dict
if isinstance(model_arch, dict) and 'model' in model_arch:
    model = model_arch['model']

# Print total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params / 1e6:.2f} million")

Total Parameters: 37.62 million


In [10]:
import numpy as np
n_qubits = int(np.ceil(np.log2(total_params)))
n_qubits

26

In [11]:
#traial with quantum train:29 April 2025
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchquantum as tq
from torch.utils.data import DataLoader


In [12]:
# CIFAR-10 setup
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [13]:
# Load YOLOv7 model (must be cloned from official GitHub and available in PYTHONPATH)
from utils.general import check_img_size
from models.experimental import attempt_load



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_dict = torch.load("yolov7.pt", map_location=device, weights_only=False)

if isinstance(model_dict, dict) and 'model' in model_dict:
    model = model_dict['model']
else:
    model = model_dict
model.eval()

model.eval()

# Extract and flatten weights
numpy_weights = {}
nw_list = []
nw_list_normal = []
for name, param in model.state_dict().items():
    numpy_weights[name] = param.cpu().numpy()
for i in numpy_weights:
    nw_list.append(list(numpy_weights[i].flatten()))
for i in nw_list:
    for j in i:
        nw_list_normal.append(j)

print("# of NN parameters: ", len(nw_list_normal))
n_qubit = 26

# of NN parameters:  37669981


In [14]:
#Here are the basic blocks of Quantum Train 
# Generate qubit states
def generate_qubit_states_torch(n_qubit, n_samples):
    return torch.randint(low=-1, high=2, size=(n_samples, n_qubit)).float()

class MappingModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super().__init__()
        layers = [nn.Linear(input_size, hidden_sizes[0]), nn.ReLU()]
        for i in range(len(hidden_sizes)-1):
            layers += [nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.ReLU()]
        layers += [nn.Linear(hidden_sizes[-1], output_size)]
        self.net = nn.Sequential(*layers)

    def forward(self, X):
        return self.net(X)

class QLayer(nn.Module):
    def __init__(self, n_blocks=4):
        super().__init__()
        self.n_qubits = n_qubit
        self.n_blocks = n_blocks
        self.u3_layers = tq.QuantumModuleList()
        self.cu3_layers = tq.QuantumModuleList()
        for _ in range(n_blocks):
            self.u3_layers.append(tq.Op1QAllLayer(op=tq.U3, n_wires=n_qubit, has_params=True, trainable=True))
            self.cu3_layers.append(tq.Op2QAllLayer(op=tq.CU3, n_wires=n_qubit, has_params=True, trainable=True, circular=True))

    def forward(self):
        qdev = tq.QuantumDevice(n_wires=self.n_qubits, bsz=1, device=device)
        for k in range(self.n_blocks):
            self.u3_layers[k](qdev)
            self.cu3_layers[k](qdev)
        state = qdev.get_states_1d().abs()[0]
        return state[:len(nw_list_normal)] ** 2

class QuantumTrainYOLOv7(nn.Module):
    def __init__(self, yolo_model, output_size):
        super().__init__()
        self.yolo = yolo_model
        self.qnn = QLayer()
        self.mapping = MappingModel(input_size=n_qubit+1, hidden_sizes=[128, 512, 256], output_size=1)

    def forward(self, x):
        probs = self.qnn()
        num_samples = 500  # this my trial to solve kernel die problem
        qubit_states = generate_qubit_states_torch(n_qubit, num_samples).repeat((len(nw_list_normal) // num_samples + 1, 1))[:len(nw_list_normal)].to(x.device)
       # qubit_states = generate_qubit_states_torch(n_qubit, len(nw_list_normal)).to(x.device)
        probs = probs[:len(nw_list_normal)].unsqueeze(1)
        input_data = torch.cat([qubit_states, probs], dim=1)
        weights = self.mapping(input_data)
        weights = weights.view(-1).detach()

        new_state_dict = {}
        data_iter = iter(weights)
        for name, param in self.yolo.state_dict().items():
            shape = param.shape
            num = param.numel()
            flat = torch.tensor([next(data_iter).item() for _ in range(num)]).view(shape)
            new_state_dict[name] = flat.type_as(param)

        self.yolo.load_state_dict(new_state_dict, strict=False)
        return self.yolo(x)



In [15]:
# Instantiate Quantum YOLOv7
quantum_yolo = QuantumTrainYOLOv7(model, output_size=1).to(device)
optimizer = torch.optim.Adam(quantum_yolo.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training loop (1 epoch for test)
for epoch in range(1):
    quantum_yolo.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        print("dedug 1")
        outputs = quantum_yolo(images)
        print("dedug 2")
        outputs = outputs.mean([2, 3])  # spatial average
        print("dedug 3")
        loss = criterion(outputs, labels)
        print("dedug 4")
        loss.backward()
        print("dedug 5")
        optimizer.step()
        print("dedug 6")
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

dedug 1


In [ ]:
# Evaluate
quantum_yolo.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = quantum_yolo(images).mean([2, 3])
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")